In [1]:
import sys
import os
import warnings
warnings.filterwarnings("ignore")

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from flexml.regression import Regression
from sklearn.datasets import load_diabetes

In [2]:
df = load_diabetes(as_frame = True)['frame']

In [3]:
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
 10  target  442 non-null    float64
dtypes: float64(11)
memory usage: 38.1 KB


In [5]:
reg_exp = Regression(df, target_col = "target")
reg_exp.start_experiment()

INFO | 2024-08-16 00:25:19 | [PROCESS] Training the ML models
100%|██████████| 6/6 [00:00<00:00,  7.36it/s]
INFO | 2024-08-16 00:25:20 | [PROCESS] Model training is finished!


,model_name,r2,mae,mse,rmse
0,LinearRegression,0.484900,41.548500,2848.310700,53.369600
1,RandomForestRegressor,0.474600,42.923600,2905.430800,53.902000
2,CatBoostRegressor,0.457800,42.523300,2998.203300,54.755900
3,LGBMRegressor,0.401900,44.063200,3307.346000,57.509500
4,XGBRegressor,0.391400,46.585700,3365.329800,58.011500
5,DecisionTreeRegressor,-0.146500,59.991000,6339.630600,79.621800


In [6]:
reg_exp.show_model_stats()

,model_name,r2,mae,mse,rmse
0,LinearRegression,0.484900,41.548500,2848.310700,53.369600
1,RandomForestRegressor,0.474600,42.923600,2905.430800,53.902000
2,CatBoostRegressor,0.457800,42.523300,2998.203300,54.755900
3,LGBMRegressor,0.401900,44.063200,3307.346000,57.509500
4,XGBRegressor,0.391400,46.585700,3365.329800,58.011500
5,DecisionTreeRegressor,-0.146500,59.991000,6339.630600,79.621800


In [7]:
best_model = reg_exp.get_best_models()

In [8]:
print(best_model)

LinearRegression()


In [9]:
reg_exp.tune_model(best_model, tuning_method="randomized_search", n_trials = 4, cv = 2)

INFO | 2024-08-16 00:25:20 | [PROCESS] Model Tuning process is started


Fitting 2 folds for each of 2 candidates, totalling 4 fits


INFO | 2024-08-16 00:25:22 | [PROCESS] Model Tuning process is finished


In [10]:
tuned_model = reg_exp.tuned_model

In [11]:
reg_exp.tuned_model_score

0.5253